In [ ]:
!pip install -q -U transformers datasets bitsandbytes peft

In [1]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

HF_TOKEN = user_secrets.get_secret("HF_TOKEN")

In [2]:
import huggingface_hub

huggingface_hub.login(token=HF_TOKEN)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
from datasets import load_dataset
dataset = load_dataset("BLACKBUN/old_korean_newspaper_1897_1910_economy_politic_qa")

In [4]:
# for 2 billion model

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from peft import PeftModel
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

BASE_MODEL = "google/gemma-2-2b-it"
ADAPTER = "jia6776/korean_history_1897_1910_gemma2_2b_lora"

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

model = PeftModel.from_pretrained(model, ADAPTER).to(device)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# for 9 billion model

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from peft import PeftModel
import torch

BASE_MODEL = "google/gemma-2-9b-it"
ADAPTER = "BLACKBUN/korean_history_1897_1910_gemma2_9b_lora_q4"

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

model = PeftModel.from_pretrained(model, ADAPTER).to(device)

In [5]:
question = dataset['test']['Question'][100]
answer = dataset['test']['Answer'][100]

In [6]:
print(f"Q:{question}")

Q:대한제국 시기 "Every-Day Life in Korea"라는 책의 출판이 가지는 역사적 의미는 무엇인가요?


In [7]:
messages = [
    # {
    #     "role": "system",
    #     "content": "You are a helpful assistant."
    # },
    {
        "role": "user",
        "content": f"{question}"
    },

]
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [8]:
input_text = prompt
inputs = tokenizer(input_text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_length=256)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [9]:
print(f"Model Answer: {generated_text}\n")
print(f"Ground Answer: {answer}")

Model Answer: user
대한제국 시기 "Every-Day Life in Korea"라는 책의 출판이 가지는 역사적 의미는 무엇인가요?
model
"Every-Day Life in Korea"는 대한제국 시기 한국인들이 겪고 있는 현실을 객관적으로 기록한 책으로, 당시 사회의 복잡한 문제와 그에 대한 인식을 보여줍니다. 이 책은 한국인들이 자국의 문화와 역사를 이해하고, 외세의 압박 속에서도 자주성을 유지하려는 노력을 반영하고 있습니다. 또한, 한국인들이 서구 문명과의 접촉을 통해 어떻게 변화하고 있었는지를 보여주는 중요한 자료로, 당시 한국 사회의 변화와 발전을 이해하는 데 기여할 수 있습니다.

Ground Answer: "Every-Day Life in Korea"는 대한제국 시기의 한국 사회와 문화를 외부에 알리는 중요한 자료로, 외국인들이 한국의 일상생활을 이해하는 데 기여했습니다. 이 책은 한국에 대한 서구의 관심을 증대시키고, 한국의 문화적 정체성을 재조명하는 계기가 되었으며, 당시 한국의 사회적, 경제적 상황을 기록함으로써 역사적 연구에 중요한 기초 자료로 활용될 수 있습니다.


In [ ]:
pipe_finetuned = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, device="cuda")